In [1]:
import datetime
from pathlib import Path
from collections import Counter

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

from model import conv_block
from data import example_to_tensor, normalize, add_channel_axis, train_test_split
from utils import plot_slice, plot_animated_volume
from config import data_root_dir

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [2]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
#
# policy = mixed_precision.Policy("mixed_float16")
##policy = mixed_precision.Policy("float32")
# mixed_precision.set_policy(policy)
# print("Compute dtype: %s" % policy.compute_dtype)
# print("Variable dtype: %s" % policy.variable_dtype)

In [3]:
downscaling = 2
if downscaling == 4:
    input_shape = (24, 128, 128, 1)
    neg_tfrecord_glob = "CT-0-0.25/*.tfrecord"
    pos_tfrecord_glob = "CT-[1-4]-0.25/*.tfrecord"
elif downscaling == 2:
    input_shape = (48, 256, 256, 1)
    neg_tfrecord_glob = "CT-0-0.5/*.tfrecord"
    pos_tfrecord_glob = "CT-[1-4]-0.5/*.tfrecord"
elif downscaling == 1:
    input_shape = (96, 512, 512, 1)
    neg_tfrecord_glob = "CT-0/*.tfrecord"
    pos_tfrecord_glob = "CT-[1-4]/*.tfrecord"
else:
    raise RuntimeError("Downscaling not supported")

epochs = 1000
patience = 20
batch_size = 4
learning_rate = 0.0001
dropout_rate = 0.0
seed = 5
val_perc = 0.12  # percentage from the already splitted training test
test_perc = 0.1

In [4]:
neg_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(neg_tfrecord_glob)]
neg_x = (
    tf.data.TFRecordDataset(neg_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_neg = sum(1 for _ in neg_x)
num_neg = 254
print(f"Number of negative samples: {num_neg}")
neg_x

Number of negative samples: 254


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [5]:
pos_tfrecord_fnames = [str(p) for p in Path(data_root_dir).glob(pos_tfrecord_glob)]
pos_x = (
    tf.data.TFRecordDataset(pos_tfrecord_fnames)
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(add_channel_axis, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_pos = sum(1 for _ in pos_x)
num_pos = 856
print(f"Number of positive samples: {num_pos}")
pos_x

Number of positive samples: 856


<ParallelMapDataset shapes: (None, None, None, 1), types: tf.float32>

In [6]:
neg_y = tf.data.Dataset.from_tensors(tf.constant([0], dtype=tf.int8)).repeat(num_neg)
neg_dataset = tf.data.Dataset.zip((neg_x, neg_y))
neg_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [7]:
pos_y = tf.data.Dataset.from_tensors(tf.constant([1], dtype=tf.int8)).repeat(num_pos)
pos_dataset = tf.data.Dataset.zip((pos_x, pos_y))
pos_dataset

<ZipDataset shapes: ((None, None, None, 1), (1,)), types: (tf.float32, tf.int8)>

In [8]:
dataset = neg_dataset.concatenate(pos_dataset)
dataset, test_dataset = train_test_split(
    dataset,
    test_perc=test_perc,
    cardinality=(num_pos + num_neg),
    seed=seed,
)
test_dataset = test_dataset.padded_batch(1, (input_shape, (1,)))
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=None,
    seed=seed,
)
val_dataset = (
    val_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.padded_batch(batch_size, (input_shape, (1,)), drop_remainder=True)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

<PrefetchDataset shapes: ((4, 48, 256, 256, 1), (4, 1)), types: (tf.float32, tf.int8)>

In [9]:
def count_labels(dataset):
    "Return a dictionary of the label count."
    return dict(Counter(label.numpy()[0] for _, label in dataset.unbatch()))


print(f"Train labels:\n\t{count_labels(train_dataset)}")
print(f"Validation labels:\n\t{count_labels(val_dataset)}")
print(f"Test labels:\n\t{count_labels(test_dataset)}")

Train labels:
	{0: 203, 1: 677}
Validation labels:
	{1: 95, 0: 21}
Test labels:
	{0: 30, 1: 81}


In [10]:
inputs = keras.layers.Input(input_shape)

x = conv_block(inputs, filters=32, dropout_rate=dropout_rate)
x = conv_block(x, filters=64, dropout_rate=dropout_rate)
x = conv_block(x, filters=128, dropout_rate=dropout_rate)

x = keras.layers.GlobalAveragePooling3D()(x)
x = keras.layers.Dense(
    512,
    kernel_initializer="lecun_normal",
    bias_initializer="lecun_normal",
    activation="selu",
)(x)
x = keras.layers.AlphaDropout(dropout_rate)(x)

outputs = keras.layers.Dense(
    1,
    activation="sigmoid",
)(x)

cnn = keras.Model(inputs, outputs, name="baseline-3dcnn")
cnn.summary()

Model: "baseline-3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 256, 256, 1)] 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 48, 256, 256, 32)  896       
_________________________________________________________________
alpha_dropout (AlphaDropout) (None, 48, 256, 256, 32)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 24, 128, 128, 32)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 24, 128, 128, 64)  55360     
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 24, 128, 128, 64)  0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 12, 64, 64, 64) 

In [11]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.TruePositives(name="tp"),
        keras.metrics.FalsePositives(name="fp"),
        keras.metrics.TrueNegatives(name="tn"),
        keras.metrics.FalseNegatives(name="fn"),
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(name="auc"),
    ],
)

In [12]:
monitor_metric = "val_auc"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/baseline-{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, mode="max", verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric, patience=patience, mode="max"
)
log_dir = f"logs/baseline-{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{seed=}; "
        f"{downscaling=}; "
        f"{epochs=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{learning_rate=}; "
        f"{dropout_rate=}; "
        f"{val_perc=}; "
        f"{test_perc=}",
        step=0,
    )
cnn = keras.models.load_model(best_checkpoint)

Epoch 1/1000
    220/Unknown - 47s 213ms/step - loss: 0.5907 - tp: 654.0000 - fp: 196.0000 - tn: 7.0000 - fn: 23.0000 - accuracy: 0.7511 - precision: 0.7694 - recall: 0.9660 - auc: 0.5086
Epoch 00001: val_auc improved from -inf to 0.50175, saving model to models/baseline-20201025-120509.h5
220/220 [==============================] - 50s 229ms/step - loss: 0.5907 - tp: 654.0000 - fp: 196.0000 - tn: 7.0000 - fn: 23.0000 - accuracy: 0.7511 - precision: 0.7694 - recall: 0.9660 - auc: 0.5086 - val_loss: 0.4789 - val_tp: 95.0000 - val_fp: 21.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8190 - val_precision: 0.8190 - val_recall: 1.0000 - val_auc: 0.5018
Epoch 2/1000
220/220 [==============================] - ETA: 0s - loss: 0.5490 - tp: 677.0000 - fp: 203.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.7693 - precision: 0.7693 - recall: 1.0000 - auc: 0.5248
Epoch 00002: val_auc improved from 0.50175 to 0.52682, saving model to models/baseline-20201025-120509.h5
220/220

Epoch 14/1000
220/220 [==============================] - ETA: 0s - loss: 0.5104 - tp: 666.0000 - fp: 199.0000 - tn: 4.0000 - fn: 11.0000 - accuracy: 0.7614 - precision: 0.7699 - recall: 0.9838 - auc: 0.6732
Epoch 00014: val_auc did not improve from 0.55013
220/220 [==============================] - 49s 225ms/step - loss: 0.5104 - tp: 666.0000 - fp: 199.0000 - tn: 4.0000 - fn: 11.0000 - accuracy: 0.7614 - precision: 0.7699 - recall: 0.9838 - auc: 0.6732 - val_loss: 0.4973 - val_tp: 95.0000 - val_fp: 21.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8190 - val_precision: 0.8190 - val_recall: 1.0000 - val_auc: 0.5361
Epoch 15/1000
220/220 [==============================] - ETA: 0s - loss: 0.5108 - tp: 669.0000 - fp: 198.0000 - tn: 5.0000 - fn: 8.0000 - accuracy: 0.7659 - precision: 0.7716 - recall: 0.9882 - auc: 0.6654
Epoch 00015: val_auc did not improve from 0.55013
220/220 [==============================] - 49s 225ms/step - loss: 0.5108 - tp: 669.0000 - fp: 198.0000 -

Epoch 27/1000
220/220 [==============================] - ETA: 0s - loss: 0.4963 - tp: 663.0000 - fp: 186.0000 - tn: 17.0000 - fn: 14.0000 - accuracy: 0.7727 - precision: 0.7809 - recall: 0.9793 - auc: 0.6980
Epoch 00027: val_auc did not improve from 0.55739
220/220 [==============================] - 49s 225ms/step - loss: 0.4963 - tp: 663.0000 - fp: 186.0000 - tn: 17.0000 - fn: 14.0000 - accuracy: 0.7727 - precision: 0.7809 - recall: 0.9793 - auc: 0.6980 - val_loss: 0.5024 - val_tp: 95.0000 - val_fp: 21.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8190 - val_precision: 0.8190 - val_recall: 1.0000 - val_auc: 0.5398
Epoch 28/1000
220/220 [==============================] - ETA: 0s - loss: 0.5001 - tp: 660.0000 - fp: 198.0000 - tn: 5.0000 - fn: 17.0000 - accuracy: 0.7557 - precision: 0.7692 - recall: 0.9749 - auc: 0.6928
Epoch 00028: val_auc did not improve from 0.55739
220/220 [==============================] - 49s 225ms/step - loss: 0.5001 - tp: 660.0000 - fp: 198.000

Epoch 40/1000
220/220 [==============================] - ETA: 0s - loss: 0.4917 - tp: 655.0000 - fp: 181.0000 - tn: 22.0000 - fn: 22.0000 - accuracy: 0.7693 - precision: 0.7835 - recall: 0.9675 - auc: 0.7053
Epoch 00040: val_auc improved from 0.57093 to 0.58120, saving model to models/baseline-20201025-120509.h5
220/220 [==============================] - 50s 227ms/step - loss: 0.4917 - tp: 655.0000 - fp: 181.0000 - tn: 22.0000 - fn: 22.0000 - accuracy: 0.7693 - precision: 0.7835 - recall: 0.9675 - auc: 0.7053 - val_loss: 0.4764 - val_tp: 95.0000 - val_fp: 21.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8190 - val_precision: 0.8190 - val_recall: 1.0000 - val_auc: 0.5812
Epoch 41/1000
220/220 [==============================] - ETA: 0s - loss: 0.4960 - tp: 659.0000 - fp: 189.0000 - tn: 14.0000 - fn: 18.0000 - accuracy: 0.7648 - precision: 0.7771 - recall: 0.9734 - auc: 0.7023
Epoch 00041: val_auc did not improve from 0.58120
220/220 [==============================] - 4

Epoch 53/1000
220/220 [==============================] - ETA: 0s - loss: 0.4889 - tp: 657.0000 - fp: 182.0000 - tn: 21.0000 - fn: 20.0000 - accuracy: 0.7705 - precision: 0.7831 - recall: 0.9705 - auc: 0.7113
Epoch 00053: val_auc did not improve from 0.59799
220/220 [==============================] - 49s 225ms/step - loss: 0.4889 - tp: 657.0000 - fp: 182.0000 - tn: 21.0000 - fn: 20.0000 - accuracy: 0.7705 - precision: 0.7831 - recall: 0.9705 - auc: 0.7113 - val_loss: 0.4915 - val_tp: 95.0000 - val_fp: 21.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8190 - val_precision: 0.8190 - val_recall: 1.0000 - val_auc: 0.5754
Epoch 54/1000
220/220 [==============================] - ETA: 0s - loss: 0.4872 - tp: 656.0000 - fp: 180.0000 - tn: 23.0000 - fn: 21.0000 - accuracy: 0.7716 - precision: 0.7847 - recall: 0.9690 - auc: 0.7147
Epoch 00054: val_auc did not improve from 0.59799
220/220 [==============================] - 49s 225ms/step - loss: 0.4872 - tp: 656.0000 - fp: 180.00

...

Epoch 118/1000
220/220 [==============================] - ETA: 0s - loss: 0.4445 - tp: 643.0000 - fp: 144.0000 - tn: 59.0000 - fn: 34.0000 - accuracy: 0.7977 - precision: 0.8170 - recall: 0.9498 - auc: 0.7824
Epoch 00118: val_auc did not improve from 0.66266
220/220 [==============================] - 49s 225ms/step - loss: 0.4445 - tp: 643.0000 - fp: 144.0000 - tn: 59.0000 - fn: 34.0000 - accuracy: 0.7977 - precision: 0.8170 - recall: 0.9498 - auc: 0.7824 - val_loss: 0.4473 - val_tp: 94.0000 - val_fp: 19.0000 - val_tn: 2.0000 - val_fn: 1.0000 - val_accuracy: 0.8276 - val_precision: 0.8319 - val_recall: 0.9895 - val_auc: 0.6551
Epoch 119/1000
220/220 [==============================] - ETA: 0s - loss: 0.4476 - tp: 647.0000 - fp: 148.0000 - tn: 55.0000 - fn: 30.0000 - accuracy: 0.7977 - precision: 0.8138 - recall: 0.9557 - auc: 0.7810
Epoch 00119: val_auc did not improve from 0.66266
220/220 [==============================] - 49s 225ms/step - loss: 0.4476 - tp: 647.0000 - fp: 148.0000 - t

Epoch 131/1000
220/220 [==============================] - ETA: 0s - loss: 0.4317 - tp: 647.0000 - fp: 139.0000 - tn: 64.0000 - fn: 30.0000 - accuracy: 0.8080 - precision: 0.8232 - recall: 0.9557 - auc: 0.8025
Epoch 00131: val_auc improved from 0.66566 to 0.68822, saving model to models/baseline-20201025-120509.h5
220/220 [==============================] - 50s 227ms/step - loss: 0.4317 - tp: 647.0000 - fp: 139.0000 - tn: 64.0000 - fn: 30.0000 - accuracy: 0.8080 - precision: 0.8232 - recall: 0.9557 - auc: 0.8025 - val_loss: 0.4425 - val_tp: 90.0000 - val_fp: 19.0000 - val_tn: 2.0000 - val_fn: 5.0000 - val_accuracy: 0.7931 - val_precision: 0.8257 - val_recall: 0.9474 - val_auc: 0.6882
Epoch 132/1000
220/220 [==============================] - ETA: 0s - loss: 0.4265 - tp: 644.0000 - fp: 145.0000 - tn: 58.0000 - fn: 33.0000 - accuracy: 0.7977 - precision: 0.8162 - recall: 0.9513 - auc: 0.8058
Epoch 00132: val_auc did not improve from 0.68822
220/220 [==============================] - 49s 225

Epoch 144/1000
220/220 [==============================] - ETA: 0s - loss: 0.4079 - tp: 643.0000 - fp: 126.0000 - tn: 77.0000 - fn: 34.0000 - accuracy: 0.8182 - precision: 0.8362 - recall: 0.9498 - auc: 0.8266
Epoch 00144: val_auc did not improve from 0.69348
220/220 [==============================] - 50s 225ms/step - loss: 0.4079 - tp: 643.0000 - fp: 126.0000 - tn: 77.0000 - fn: 34.0000 - accuracy: 0.8182 - precision: 0.8362 - recall: 0.9498 - auc: 0.8266 - val_loss: 0.4369 - val_tp: 91.0000 - val_fp: 18.0000 - val_tn: 3.0000 - val_fn: 4.0000 - val_accuracy: 0.8103 - val_precision: 0.8349 - val_recall: 0.9579 - val_auc: 0.6930
Epoch 145/1000
220/220 [==============================] - ETA: 0s - loss: 0.4142 - tp: 644.0000 - fp: 124.0000 - tn: 79.0000 - fn: 33.0000 - accuracy: 0.8216 - precision: 0.8385 - recall: 0.9513 - auc: 0.8215
Epoch 00145: val_auc did not improve from 0.69348
220/220 [==============================] - 49s 225ms/step - loss: 0.4142 - tp: 644.0000 - fp: 124.0000 - t

Epoch 157/1000
220/220 [==============================] - ETA: 0s - loss: 0.3901 - tp: 649.0000 - fp: 121.0000 - tn: 82.0000 - fn: 28.0000 - accuracy: 0.8307 - precision: 0.8429 - recall: 0.9586 - auc: 0.8458
Epoch 00157: val_auc did not improve from 0.69900
220/220 [==============================] - 50s 225ms/step - loss: 0.3901 - tp: 649.0000 - fp: 121.0000 - tn: 82.0000 - fn: 28.0000 - accuracy: 0.8307 - precision: 0.8429 - recall: 0.9586 - auc: 0.8458 - val_loss: 0.4617 - val_tp: 89.0000 - val_fp: 15.0000 - val_tn: 6.0000 - val_fn: 6.0000 - val_accuracy: 0.8190 - val_precision: 0.8558 - val_recall: 0.9368 - val_auc: 0.6830
Epoch 158/1000
220/220 [==============================] - ETA: 0s - loss: 0.3917 - tp: 646.0000 - fp: 122.0000 - tn: 81.0000 - fn: 31.0000 - accuracy: 0.8261 - precision: 0.8411 - recall: 0.9542 - auc: 0.8418
Epoch 00158: val_auc did not improve from 0.69900
220/220 [==============================] - 49s 225ms/step - loss: 0.3917 - tp: 646.0000 - fp: 122.0000 - t

Epoch 170/1000
220/220 [==============================] - ETA: 0s - loss: 0.3789 - tp: 635.0000 - fp: 105.0000 - tn: 98.0000 - fn: 42.0000 - accuracy: 0.8330 - precision: 0.8581 - recall: 0.9380 - auc: 0.8534
Epoch 00170: val_auc did not improve from 0.72682
220/220 [==============================] - 49s 225ms/step - loss: 0.3789 - tp: 635.0000 - fp: 105.0000 - tn: 98.0000 - fn: 42.0000 - accuracy: 0.8330 - precision: 0.8581 - recall: 0.9380 - auc: 0.8534 - val_loss: 0.4486 - val_tp: 93.0000 - val_fp: 19.0000 - val_tn: 2.0000 - val_fn: 2.0000 - val_accuracy: 0.8190 - val_precision: 0.8304 - val_recall: 0.9789 - val_auc: 0.7093
Epoch 171/1000
220/220 [==============================] - ETA: 0s - loss: 0.3831 - tp: 643.0000 - fp: 111.0000 - tn: 92.0000 - fn: 34.0000 - accuracy: 0.8352 - precision: 0.8528 - recall: 0.9498 - auc: 0.8512
Epoch 00171: val_auc did not improve from 0.72682
220/220 [==============================] - 49s 225ms/step - loss: 0.3831 - tp: 643.0000 - fp: 111.0000 - t

In [ ]:
# cnn = keras.models.load_model("models/baseline-20201025-005657.h5")

In [15]:
cnn.evaluate(test_dataset, verbose=1, return_dict=True)

220/220 [==============================] - 16s 71ms/step - loss: 0.3772 - tp: 663.0000 - fp: 129.0000 - tn: 74.0000 - fn: 14.0000 - accuracy: 0.8375 - precision: 0.8371 - recall: 0.9793 - auc: 0.8713


{'loss': 0.3771935999393463,
 'tp': 663.0,
 'fp': 129.0,
 'tn': 74.0,
 'fn': 14.0,
 'accuracy': 0.8374999761581421,
 'precision': 0.8371211886405945,
 'recall': 0.9793205261230469,
 'auc': 0.8713499903678894}

In [ ]:
x, y = next(iter(test_dataset.skip(0)))
prediction = cnn(x, training=False)
print(f"real: {y.numpy()}, prediction: {prediction.numpy()}")
plot_animated_volume(x[0, :], fps=2)

In [17]:
def prediction_bias(dataset):
    """Prediction bias is the difference
        average_labels - average_predictions
    
    It should be near zero.
    Return the tuple (label_avg, prediction_avg, prediction_bias)
    """
    label_avg = np.mean([label.numpy()[0] for _, label in dataset.unbatch()])

    def gen():
        for x, _ in dataset:
            yield x

    x_dataset = (
        tf.data.Dataset.from_generator(gen, tf.float32)
        .unbatch()
        .padded_batch(1, input_shape)
    )
    prediction_avg = np.mean([cnn(x, training=False).numpy()[0][0] for x in x_dataset])
    return label_avg, prediction_avg, np.abs(label_avg - prediction_avg)

In [20]:
l, p, b = prediction_bias(test_dataset)
print(f"Labels average: {l}")
print(f"Predictions average: {p}")
print(f"Prediction bias: {b}")

Labels average: 0.7297297297297297
Predictions average: 0.8031964898109436
Prediction bias: 0.07346676008121389
